# Crypto token correlation heatmap

In [1]:
# ! pip install -r requirements.txt

##### Fetching Token Data from CoinGecko

In [2]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"

url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": vs_currency,
    "days": days
}
headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": api_key
}

response = requests.get(url, headers=headers, params=params)
print(response.text)

{"prices":[[1751983245426,109073.07144728328],[1751987026192,108406.1743477702],[1751990639516,108269.04717656305],[1751994208603,108416.67394108746],[1751997846596,108941.07172577523],[1752001445284,109116.14147377446],[1752005042558,108784.86962448058],[1752008622474,108675.35707886492],[1752012184038,108930.28353025016],[1752015838904,108920.19114415902],[1752019436139,108945.70903639354],[1752023035281,108962.4725608281],[1752026647713,108795.96256571825],[1752030196564,108649.69348045423],[1752033802905,108382.60401320309],[1752037370113,108542.06540414323],[1752040993182,108775.99214179988],[1752044636700,108835.8966583524],[1752048124359,108712.30310967196],[1752051627281,108699.69913613351],[1752055255077,108769.59937353026],[1752059005652,108873.6901631303],[1752062606517,109082.59555008526],[1752066037275,109433.07396636377],[1752069838442,109416.18664111895],[1752073439773,108843.27368623985],[1752076997051,109069.47853205416],[1752080604519,109408.0230263172],[1752084231404

##### Converting result to a DataFrame

In [3]:
data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

                timestamp          price
0 2025-07-08 14:00:45.426  109073.071447
1 2025-07-08 15:03:46.192  108406.174348
2 2025-07-08 16:03:59.516  108269.047177
3 2025-07-08 17:03:28.603  108416.673941
4 2025-07-08 18:04:06.596  108941.071726


##### Creating Function that will enable us to be able to check the correlation of 2 or more tokens

In [4]:
# ! pip install joblib

In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib

load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"

def fetch_token_price_history(token_ids, vs_currency="usd", days="30"):
    """
    Fetch historical price data for multiple tokens from CoinGecko Pro API.
    
    Args:
        token_ids (list): List of token ids (e.g., ['bitcoin', 'ethereum']).
        vs_currency (str): The target currency (default 'usd').
        days (str): Number of days to fetch (default '30').
        
    Returns:
        pd.DataFrame: DataFrame with timestamps as index and tokens as columns.
    """
        
    load_dotenv()
    api_key = os.getenv("GECKO_API")
    price_dfs = []

    for coin_id in token_ids:
        url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
        params = {
            "vs_currency": vs_currency,
            "days": days
        }
        headers = {
            "accept": "application/json",
            "x-cg-pro-api-key": api_key
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.resample('1h').last()  # or '1D' for daily
        price_dfs.append(df)

#merge all df on timestamp
    result_df = pd.concat(price_dfs, axis=1)
    return result_df
        

In [8]:
# Example: Get tokens from user input
tokens = input("Enter token IDs separated by commas (e.g., bitcoin,ethereum,solana): ").split(",")
tokens = [t.strip() for t in tokens]

try:
	df_prices = fetch_token_price_history(tokens, days="30")
	print(df_prices.head())
except KeyError as e:
	print(f"Error: Missing key in API response - {e}")
	print("Please check if the token IDs are correct and try again.")
except Exception as e:
	print(f"An unexpected error occurred: {e}")

# Save df_prices using joblib
save_dir = r"C:\Users\Jo$h\Desktop\Visual Studio Code\Token_heatmap\token_heatmap\Database"
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

                           bitcoin     ethereum      solana    ripple
timestamp                                                            
2025-07-08 14:00:00  109073.071447  2591.368899  152.116876  2.295713
2025-07-08 15:00:00  108406.174348  2564.814872  150.877537  2.282936
2025-07-08 16:00:00  108269.047177  2564.706990  149.886831  2.285550
2025-07-08 17:00:00  108416.673941  2576.406339  150.457441  2.290661
2025-07-08 18:00:00  108941.071726  2615.922574  152.059165  2.307880


['C:\\Users\\Jo$h\\Desktop\\Visual Studio Code\\Token_heatmap\\token_heatmap\\Database\\df_prices.joblib']

In [ ]:
# This project provides framework for analyzing and visualizing the correlation between multiple crypto tokens